# Chapter 7  Markov Chain Monte Carlo

### text: Doing Bayesian Data Analysis

This chapter is about methods for approximating a posterior distribution by collecting from it a large
representative sample.   
These methods are important because complex posterior distributions are otherwise very challenging to get a handle on.

## Terminology

**Monte Carlo simulation**  
Any simulation that samples a lot of random
values from a distribution.    
Monte Carlo(모나코에 있는 도박장)

<br><br>
<img style="float: left;" src="pic3/07_A.png"  width="300">

<br><br>
<img style="float: left;" src="pic3/07_B.png"  width="300">

The **Metropolis algorithm** and **Gibbs sampling** are specific types of Monte Carlo
process.   
They generate random walks such that each step in the walk is completely
independent of the steps before the current position. 

**Markov process**  
any such process in which each step
has no memory for states before the current one 

**Markov chain**  
a succession of such Markov process steps 
It was named after the mathematician AndreyMarkov (1856–1922)

The Metropolis algorithm and Gibbs sampling are examples of a
**Markov chain Monte Carlo (MCMC)** process.   
There are many others. 

It is the invention of MCMC algorithms, along with software for automatically creating samplers for complex
models (such as Pymc3 and Stan), and fast cheap computers, that allow us to do
Bayesian data analysis for complex realistic data.

<br><br><br>

This chapter introduces the methods we will use for producing accurate approximations to Bayesian posterior distributions for realistic applications.   
The class of methods is
called Markov chain Monte Carlo (MCMC).   
It is MCMC algorithms and software, along with fast computer hardware,
that allow us to do Bayesian data analysis for realistic applications that would have been
effectively impossible 30 years ago.  


### grid approximation

We already recognized the possibility that our prior beliefs about $\theta$ could not be adequately
represented by any function that yields an analytically solvable
posterior function.   
Grid approximation is one approach to addressing such situations.
When we have just one parameter with a finite range, then approximation by a grid is
a useful procedure.  
But what if we have several parameters? 
It is much more typical to have models involving several parameters.   
In these situations, the parameter space cannot be spanned by a grid with a reasonable number of
points.   
Consider, for example, a model with six parameters.   
The parameter space is a six-dimensional space that involves the joint distribution of all combinations of
parameter values.   
If we set up a comb on each parameter that has 1,000 values, then the
six-dimensional parameter space has $1,000^6$ = 1,000,000,000,000,000,000 combinations
of parameter values, which is too many for any computer to evaluate.   
In anticipation of
those situations when grid approximation will not work, we explore a new method called
Markov chain Monte Carlo, or MCMC for short.   
In this chapter, we apply MCMC to
the simple situation of estimating a single parameter. In real research you would probably
not want to apply MCMC to such simple one-parameter models, instead going with
mathematical analysis or grid approximation.   
But it is very useful to learn about MCMC
in the one-parameter context.

<br><br>

The method described in this chapter assumes that the prior distribution is specified by a function that is easily evaluated.   
This simply means that if you specify a value for
$\theta$, then the value of $p(\theta)$ is easily determined, especially by a computer.   
The method also assumes that the value of the likelihood function, $p(D|\theta)$, can be computed for any specified values of $D$ and $\theta$. (Actually, all that the method really demands is that the
prior and likelihood can be easily computed up to a multiplicative constant.)   
In other words, the method does not require evaluating the difficult integral in the denominator of Bayes’ rule.  
What the method produces for us is an approximation of the posterior distribution, $p(\theta|D)$, in the form of a large number of $\theta$ values sampled from that distribution.
This heap of representative $\theta$ values can be used to estimate the central tendency
of the posterior, its highest density interval (HDI), etc.   
The posterior distribution is
estimated by randomly generating a lot of values from it, and therefore, by analogy
to the random events at games in a casino, this approach is called a Monte Carlo
method.

## 7.1. APPROXIMATING A DISTRIBUTION WITH A LARGE SAMPLE

<br><br>
<img style="float: left;" src="pic3/07_01.png"  width="700">

<br><br><br>

## 7.2. A SIMPLE CASE OF THE METROPOLIS ALGORITHM

Our goal in Bayesian inference is to get an accurate representation of the posterior distribution.   
One way to do that is to sample a large number of representative points
from the posterior.   
The question then becomes this: How can we sample a large number
of representative values from a distribution?   
For an answer, let’s ask a politician.

### 7.2.1.  A politician stumbles upon the Metropolis algorithm

Suppose an elected politician lives on a long chain of islands.   
He is constantly traveling from island to island, wanting to stay in the public eye.  
At the end of every day, he has to decide whether to   
(i) stay on the current island  
(ii) move to the adjacent island to the west   
(iii) move to the adjacent island to the east.  

His goal is to visit all the islands proportionally to their relative population, so
that he spends the most time on the most populated islands, and proportionally less time
on the less populated islands.   
Unfortunately, he holds his office despite having no idea
what the total population of the island chain is, and he doesn’t even know exactly how
many islands there are!   
His entourage of advisers is capable of some minimal information
gathering abilities, however.   
When they are not busy fundraising, they can ask the mayor
of the island they are on how many people are on the island. And, when the politician
proposes to visit an adjacent island, they can ask the mayor of that adjacent island how
many people are on that island.  

The politician has a simple heuristic for deciding whether to travel to the proposed island:   
First, he flips a (fair) coin to decide whether to propose the adjacent island to the east or the adjacent island to the west.   
If the proposed island has a larger population than
the current island, then he definitely goes to the proposed island.  
On the other hand,
if the proposed island has a smaller population than the current island, then he goes to
the proposed island only probabilistically, to the extent that the proposed island has a
population as big as the current island.   
If the population of the proposed island is only
half as big as the current island, the probability of going there is only 50%.  

In more detail, denote the population of the proposed island as $P_{proposed}$, and the
population of the current island as $P_{current}$.   
Then he moves to the less populated island
with probability $p_{move}=\frac{P_{proposed}}{P_{current}}$.   
The politician does this by spinning a fair spinner
marked on its circumference with uniform values from zero to one.   
If the pointed-to
value is between zero and $p_{move}$, then he moves.  

What’s amazing about this heuristic is that it works: In the long run, the probability
that the politician is on any one of the islands exactly matches the relative population of
the island!

### 7.2.2. A random walk

Suppose that there are seven islands in the chain, with relative populations as shown in the bottom panel of Figure 7.2.    

<br><br>
<img style="float: left;" src="pic3/07_02.png"  width="450">

The islands are indexed by the value $\theta$, whereby the leftmost, western island is $\theta$ = 1 and the rightmost, eastern island is $\theta$ = 7.   
The relative populations of the islands
increase linearly such that $P(\theta)$ = $\theta$.   
Notice that uppercase $P( )$ refers to the relative
population of the island, not its absolute population and not its probability mass.   

The middle panel of Figure 7.2 shows one possible trajectory taken by the politician. 
Each day corresponds to one-time increment, indicated on the vertical axis.   
The plot of the trajectory shows that on the first day (t = 1), the politician happens to start on the middle island in the chain, hence $\theta_{current}$=4.   
To decide where to go on the second day, he
flips a coin to propose moving either one position left or one position right.   
In this case,
the coin proposed moving right, hence $\theta_{proposed} = 5$.   
Because the relative population at
the proposed position is greater than the relative population at the current position (i.e., $P(5) > P(4)$), the proposed move is accepted. The trajectory shows this move, because
when $t = 2$, then $\theta = 5$.

Consider the next day, when $t = 2$ and $\theta = 5$.  
The coin flip proposes moving to the left.   
The probability of accepting this proposal is $p_{move}=\frac{P_{proposed}}{P_{current}} = \frac 4 5
= 0.80$.   
The politician then spins a fair spinner that has a circumference marked from 0
to 1, which happens to come up with a value greater than 0.80.   
Therefore, the politician
rejects the proposed move and stays at the current island.   
Hence the trajectory shows that $\theta$ is still 5 when $t = 3$.   
The middle panel of Figure 7.2 shows the trajectory for the
first 500 steps in this random walk across the islands.   
The scale of the time step is plotted
logarithmically so you can see the details of the early steps but also see the trend of the
later steps.   
There are many thousands of steps in the simulation.  
The upper panel of Figure 7.2 shows a histogram of the frequencies with which each
position is visited during this simulation.  
Notice that the sampled relative frequencies closely
mimic the actual relative populations in the bottom panel!   
In fact, a sequence generated
this way will converge, as the sequence gets longer, to an arbitrarily close approximation
of the actual relative probabilities.

### 7.2.3. General properties of a random walk

The trajectory shown in Figure 7.2 is just one possible sequence of positions when the
movement heuristic is applied.   
At each time step, the direction of the proposed move is random, and if the relative probability of the proposed position is less than that of the current position, then acceptance of the proposed move is also random.   
Because of the randomness, if the process were started over again, then the specific trajectory would
almost certainly be different.   
Regardless of the specific trajectory, in the long run the relative frequency of visits mimics the target distribution.

<br><br>
<img style="float: left;" src="pic3/07_03.png"  width="600">

Figure 7.3 shows the probability of being in each position as a function of time.  
At
time t = 1, the politician starts at $\theta = 4$.     
This starting position is indicated in the upperleft panel of Figure 7.3, labeled $t = 1$, by the fact that there is 100% probability of being at $\theta = 4$.

We want to derive the probability of ending up in each position at the next time step.
To determine the probabilities of positions for time $t = 2$, consider the possibilities from
the movement process.   
The process starts with the flip of a fair coin to decide which
direction to propose moving.   
There is a 50% probability of proposing to move right,
to $\theta = 5$.  
By inspecting the target distribution of relative probabilities in the lower-right
panel of Figure 7.3, you can see that $P(\theta=5) > P(\theta=4)$, and therefore, a rightward move
is always accepted whenever it is proposed.   
Thus, at time $t = 2$, there is a 50% probability
of ending up at $\theta = 5$.   
The panel labeled $t = 2$ in Figure 7.3 plots this probability as a
bar of height 0.5 at $\theta = 5$.  
The other 50% of the time, the proposed move is to the left, to $\theta = 3$.  
By inspecting the target distribution of relative probabilities in the lower-right
panel of Figure 7.3, you can see that $P(\theta=3) = 3$, whereas $P(\theta=4) = 4$, and therefore,
a leftward move is accepted only 3/4 of the times it is proposed. Hence, at time $t = 2$, the probability of ending up at θ = 3 is 50%·3/4 = 0.375.   
The panel labeled $t = 2$ in Figure 7.3 shows this as a bar of height 0.375 at $\theta  = 3$.  
Finally, if a leftward move is
proposed but not accepted, we just stay at $\theta  = 5$.  
The probability of this happening is
only 50%·(1 − 3/4) = 0.125.

This process repeats for the next time step. I won’t go through the arithmetic details
for each value of $\theta$.   
But it is important to notice that after two proposed moves, i.e., when
$t = 3$, the politician could be at any of the positions $\theta = 2$ through $\theta = 6$, but not yet at
$\theta = 1$ or $\theta = 7$, because he could be at most two positions away from where he started.  
The probabilities continue to be computed the same way at every time step.   
You can
see that in the early time steps, the probability distribution is not a straight incline like the target distribution.  
Instead, the probability distribution has a bulge over the starting position.   
As you can see in Figure 7.3, by time $t = 99$, the position probability is virtually
indistinguishable from the target distribution, at least for this simple distribution.   
More complex distributions require a longer duration to achieve a good approximation to the target.

The graphs of Figure 7.3 show the probability that the moving politician is at each
value of $\theta$.   
But remember, at any given time step, the politician is at only one particular
position, as shown in Figure 7.2.   
To approximate the target distribution, we let the politician meander around for many time steps while we keep track of where he has been.  
When we have a long record of where the traveler has been, we can approximate
the target probability at each value of $\theta$ by simply counting the relative number times
that the traveler visited that value.

#### summary of the algorithm for moving from one position to another

We are currently at position $\theta_{current}$.   
We then propose to move one position right or one position left.   
The specific proposal is determined by flipping a coin, which can result in
50% heads (move right) or 50% tails (move left).   
The range of possible proposed moves,
and the probability of proposing each, is called the proposal distribution.   
In the present algorithm, the proposal distribution is very simple: It has only two values with 50-50
probabilities.

Having proposed a move, we then decide whether or not to accept it.   
The acceptance decision is based on the value of the target distribution at the proposed position, relative
to the value of the target distribution at our current position.   
Specifically, if the target distribution is greater at the proposed position than at our current position, then we
definitely accept the proposed move: We always move higher if we can.   
On the other hand, if the target position is less at the proposed position than at our current position,
we accept the move probabilistically: We move to the proposed position with probability
$p_{move}=\frac{P_{proposed}}{P_{current}}$, where $P(\theta)$ is the value of the target distribution at $\theta$.   
We can combine these two possibilities, of the target distribution being higher or lower
at the proposed position than at our current position, into a single expression for the
probability of moving to the proposed position:


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\large p_{move}= min(\frac{P(\theta_{proposed})}{P(\theta_{current})},1)$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(7.1)

Notice that Equation 7.1 says that when $P(\theta_{proposed}) > P(\theta_{current})$, then $p_{move} = 1$.   
Notice also that the target distribution, $P(\theta)$, does not need to be normalized, which means
it does not need to sum to 1 as a probability distribution must.   
This is because what
matters for our choice is the ratio, $\frac{P_{proposed}}{P_{current}}$, not the absolute magnitude of
$P(\theta)$.   
This property was used in the example of the island-hopping politician: The target
distribution was the population of each island, not a normalized probability.

Having proposed a move by sampling from the proposal distribution and having
then determined the probability of accepting the move according to Equation 7.1, we
then actually accept or reject the proposed move by sampling a value from a uniform
distribution over the interval [0, 1].   
If the sampled value is between 0 and pmove, then we actually make the move.   
Otherwise, we reject the move and stay at our current position.  
The whole process repeats at the next time step

### 7.2.4. Why we care

Notice what we must be able to do in the random-walk process:  
• We must be able to generate a random value from the proposal distribution, to create
$\theta_{proposed}$.  
• We must be able to evaluate the target distribution at any proposed position, to
compute $\frac{P_{proposed}}{P_{current}}$.  
• We must be able to generate a random value from a uniform distribution, to accept
or reject the proposal according to $p_{move}$.

By being able to do those three things, we are able to do indirectly something we could not do directly: We can generate random samples from the target distribution.  
Moreover, we can generate those random samples from the target distribution even when
the target distribution is not normalized.

This technique is profoundly useful when the target distribution $P(\theta)$ is a posterior proportional to $p(D|\theta)p(\theta)$.   
Merely by evaluating $p(D|\theta)p(\theta)$, without normalizing it
by $p(D)$, we can generate random representative values from the posterior distribution.  
This result is wonderful because the method obviates direct computation of the evidence $p(D)$, which is one of the most difficult aspects of Bayesian
inference.   
By using MCMC techniques, we can do Bayesian inference in rich and
complex models.  
It has only been with the development of MCMC algorithms and
software that Bayesian inference is applicable to complex data analysis, and it has only been with the production of fast and cheap computer hardware that Bayesian inference is accessible to a wide audience

### 7.2.5. Why it works

To get an intuition for why this algorithm works, consider two adjacent positions and the probabilities of moving from one to the other.   
We’ll see that the relative transition probabilities, between adjacent positions, exactly match the relative values of the target
distribution.  
Extrapolate that result across all the positions, and you can see that, in
the long run, each position will be visited proportionally to its target value.   

Now the details:   
Suppose we are at position $\theta$.   
The probability of moving to $\theta + 1$, denoted
$p(\theta → \theta + 1)$, is the probability of proposing that move times the probability of
accepting it if proposed, which is $p(\theta → \theta + 1)=0.5·min(\frac{P(\theta+1)}{P(\theta)},1)$.    
On the other hand, if we are presently at position $\theta + 1$, the probability of moving to $\theta$ is
the probability of proposing that move times the probability of accepting it if proposed,
which is $p(\theta+1 → \theta )=0.5·min(\frac{P(\theta)}{P(\theta+1)},1)$.   
The ratio of the transition probabilities is

<img style="float: left;" src="pic3/07_04_01.png"  width="600">

Equation 7.2 tells us that during transitions back and forth between adjacent positions, the relative probability of the transitions exactly matches the relative values of the target distribution.   
That might be enough for you to get the intuition that, in the
long run, adjacent positions will be visited proportionally to their relative values in the
target distribution.   
If that’s true for adjacent positions, then, by extrapolating from one
position to the next, it must be true for the whole range of positions.

To make that intuition more defensible, we have to fill in some more details.   
To do this, I’ll use matrix arithmetic.

<img style="float: left;" src="pic3/07_04_02.png"  width="600">

The usefulness of putting the transition probabilities into a matrix is that we can then use matrix multiplication to get from any current location to the probability of the next
locations.   
Here’s a reminder of how matrixmultiplication operates.   
Consider a matrix $T$.  
The value in its $r$th row and $c$th column is denoted $T_{rc}$.  
We can multiply the matrix on its left side by a row vector $w$, which yields another row vector.   
The $c$th component of the product $wT$ is $\sum_r w_rT_{rc}$.   
In other words, to compute the $c$th component of the result,
take the row vector $w$ and multiply its components by the corresponding components
in the $c$th column of $T$, and sum up those component products. 

To use the transition matrix in Equation 7.3, we put the current location probabilities into a row vector, which I will denote $w$ because it indicates where we are.   
For example, if at the current time, we are definitely in location $\theta =4$, then $w$ has 1.0 in its $\theta =4$ component, and zeros everywhere else.   
To determine the probability of the locations at the next time step, we simply multiply $w$ by $T$.   
Here’s a key example to think through:  
When $w =[. . . , 0, 1, 0, . . .]$ with a 1 only in the $\theta$ position, then $wT$ is simply the row of $T$ corresponding to $\theta$, because the $c$th component of $wT$ is $\sum_r w_rT_{rc} = T_{\theta c}$, where
I’m using the subscript $\theta$ to stand for the index that corresponds to value $\theta$.

Matrix multiplication is a very useful procedure for keeping track of position probabilities.   
At every time step, we just multiply the current position probability vector
$w$ by the transition probability matrix $T$ to get the position probabilities for the next time step.   
We keep multiplying by $T$, over and over again, to derive the long-run position
probabilities.   
This process is exactly what generated the graphs in Figure 7.3.

*Here’s the climactic implication:   
When the vector of position probabilities is the target
distribution, it stays that way on the next time step!   
In other words, the position probabilities
are stable at the target distribution.* 

We can actually prove this result without much trouble.   
Suppose the current position probabilities are the target probabilities, i.e., 
$w =[. . . ,P(\theta-1), P(\theta), P(\theta+1), . . .]/Z$, where $Z =\sum_\theta P(\theta)$ is the normalizer for
the target distribution.   
Consider the $\theta$ component of $wT$.   
We will demonstrate that the
$\theta$ component of $wT$ is the same as the $\theta$ component of $w$, for any component $\theta$.   
The $\theta$ component of $wT$ is  $\sum_r w_rT_{r \theta}$.  
Look back at the transition matrix in Equation 7.3, and
you can see then that the $\theta$ component of $wT$ is

<img style="float: left;" src="pic3/07_04_03.png"  width="600">

To simplify that equation, we can consider separately the four cases:   
Case 1: $P(\theta) > P(\theta −1)$ and $P(\theta) > P(\theta +1)$;   
Case 2: $P(\theta) > P(\theta −1)$ and $P(\theta) < P(\theta +1)$;  
Case 3: $P(\theta) < P(\theta −1)$ and $P(\theta) > P(\theta +1)$;  
Case 4: $P(\theta) < P(\theta −1)$ and $P(\theta) < P(\theta +1)$;  

In each case, Equation 7.4 simplifies to $P(\theta)/Z$.   
For example, consider  
Case 1: $P(\theta) > P(\theta −1)$ and $P(\theta) > P(\theta +1)$.   
Equation 7.4 becomes

<img style="float: left;" src="pic3/07_04_04.png"  width="600">

If you work through the other cases, you’ll find that it always reduces to $P(\theta)/Z$.   
In conclusion, when the $\theta$ component starts at $P(\theta)/Z$, it stays at $P(\theta)/Z$.

## 7.3. THE METROPOLIS ALGORITHM MORE GENERALLY

In the previous
section, we considered the simple case of   
(i) discrete positions,   
(ii) on one dimension,
and (iii) with moves that proposed just one position left or right.   

That simple situation made it relatively easy (believe it or not) to understand the procedure and how it works.  
The general algorithm applies to   
(i) continuous values,   
(ii) on any number of dimensions,  
and (iii) with more general proposal distributions.

The essentials of the general method are the same as for the simple case.   First, we have some target distribution, $P(\theta)$, over a multidimensional continuous parameter space
from which we would like to generate representative sample values.   
We must be able
to compute the value of  $P(\theta)$ for any candidate value of  $\theta$.   The distribution,  $P(\theta)$,
does not have to be normalized, however.   
It merely needs to be nonnegative.   
In typical
applications,  $P(\theta)$ is the unnormalized posterior distribution on $\theta$, which is to say, it is
the product of the likelihood and the prior.

Sample values from the target distribution are generated by taking a random walk through the parameter space.   
The walk starts at some arbitrary point, specified by the user. The starting point should be someplace where $P(\theta)$ is nonzero.   
The random walk progresses at each time step by proposing a move to a new position in parameter space
and then deciding whether or not to accept the proposed move.   
Proposal distributions can take on many different forms, with the goal being to use a proposal distribution
that efficiently explores the regions of the parameter space where $P(\theta)$ has most of its mass.   
Of course, we must use a proposal distribution for which we have a quick way to generate random values!  
For our purposes,we will consider the generic case in which the proposal distribution is normal, centered at the current position.  
The idea behind using a normal distribution is that the proposed move will typically be near the current position,
with the probability of proposing a more distant position dropping off according to the normal curve.   
Computer languages such as Python have built-in functions for generating
pseudorandom values from a normal distribution.   

Having generated a proposed new position, the algorithm then decides whether or not to accept the proposal.  
The decision rule is exactly what was already specified in Equation 7.1.   
In detail, this is accomplished by computing the ratio $p_{move} =\frac{P(\theta_{proposed})}{P(\theta_{current})}$.  
Then a random number from the uniform interval [0, 1] is generated.  
If the random number is between 0 and $p_{move}, then the move is accepted.   
The process repeats and, in the long run, the positions visited by the random walk will closely approximate the
target distribution.

### 7.3.1. Metropolis algorithmapplied to Bernoulli likelihood and beta prior

In the scenario of the island-hopping politician, the islands represented candidate parameter values, and the relative populations represented relative posterior probabilities.  
In the scenario of coin flipping, the parameter $\theta$ has values that range on a continuum from zero to one, and the relative posterior probability is computed as likelihood times prior.   
To apply the Metropolis algorithm, we conceive of the parameter dimensionas a dense chain of infinitesimal islands, and we think of the (relative) population of each infinitesimal island as its (relative) posterior probability density.   
And, instead of the proposed jump being only to immediately adjacent islands, the proposed jump can be to islands farther away from the current island.  
We need a proposal distribution that will let
us visit any parameter value on the continuum.   
For this purpose, we will use the familiar normal distribution. 

We will apply the Metropolis algorithm to the following familiar scenario.   We flip a coin N times and observe z heads.   
We use a Bernoulli likelihood function,
$p(z,N|\theta)=\theta^z (1−\theta)^{(N−z)}$.  
We start with a prior $p(\theta) = beta(\theta|a, b)$.   
For the proposed jump in the Metropolis algorithm, we will use a normal distribution centered at zero with standard deviation (SD) denoted as $\sigma$.  
We denote the proposed jump as $\Delta\theta ∼
normal(\mu=0, \sigma)$, where the symbol “$∼$” means that the value is randomly sampled from the distribution.   
Thus, the proposed jump is usually close to the current position, because the mean jump is zero, but the proposed jump can be positive or negative, with larger magnitudes less likely than smaller magnitudes.   
Denote the current parameter value as $\theta_{cur}$ and the proposed parameter value as $\theta_{pro} = \theta_{cur}+\Delta\theta$.

The Metropolis algorithm then proceeds as follows.   
Start at an arbitrary initial value of $\theta$ (in the valid range).  
This is the current value, denoted $\theta_{cur}$.   
Then:  
**1.** Randomly generate a proposed jump, $\Delta\theta ∼
normal(\mu=0, \sigma)$ and denote the
proposed value of the parameter as $\theta_{pro} = \theta_{cur}+\Delta\theta$.  
**2.** Compute the probability of moving to the proposed value as Equation 7.1, specifically expressed here as

<img style="float: left;" src="pic3/07_04_05.png"  width="600">

If the proposed value $\theta_{pro}$ happens to land outside the permissible bounds of $\theta$, the
prior and/or likelihood is set to zero, hence $p_{move}$ is zero.

**3.** Accept the proposed parameter value if a random value sampled froma [0, 1] uniform
distribution is less than $p_{move}$, otherwise reject the proposed parameter value and tally
the current value again.

Repeat the above steps until it is judged that a sufficiently representative sample has been generated.   
The judgment of “sufficiently representative” is not trivial and is an issue that will be discussed later in this chapter.   
For now, the goal is to understand the mechanics of the Metropolis algorithm.

Figure 7.4 shows specific examples of the Metropolis algorithm applied to a case with a $beta(\theta|1, 1)$ prior, N =20, and z=14.   
There are three columns in Figure 7.4, for three different runs of the Metropolis algorithm using three different values for $\sigma$ in
the proposal distribution.   
In all three cases, $\theta$ was arbitrarily started at 0.01, merely for
purposes of illustration.

<img style="float: left;" src="pic3/07_05.png"  width="700">

The middle column of Figure 7.4 uses a moderately sized SD for the proposal
distribution, namely $\sigma =0.2$, as indicated in the title of the upper-middle panel where
it says “Prpsl.SD=0.2.”   
This means that at any step in the chain, for whatever value $\theta$
happens to be, the proposed jump is within ±0.2 of $\theta$ about 68% of the time (because a
normal distribution has about 68% of its mass between −1 and +1 SDs).   
In this case, the proposed jumps are accepted roughly half the time, as indicated in the center panel by the
annotation $N_{acc}/N_{pro} =0.495$, which is the number of accepted proposals divided by the total number of proposals in the chain. This setting of the proposal distribution allows
the chain to move around parameter space fairly efficiently. In particular, you can see
in the lower-middle panel that the chain moves away quickly from the unrepresentative
starting position of $\theta = 0.01$.   
And, the chain visits a variety of representative values in
relatively few steps. That is, the chain is not very clumpy.   
The upper-middle panel shows
a histogram of the chain positions after 50,000 steps.   
The histogram looks smooth and is
an accurate approximation of the true underlying posterior distribution, which we know in this case is a $beta(\theta|15, 7)$ distribution.   
Although the chain is not very clumpy and yields a smooth-looking histogram, it does have some clumpiness because each step is linked to the location of the previous step, and about half the steps don’t change at all.   
Thus, there are not 50,000 independent representative values of the
posterior distribution in the chain.   
If we take into account the clumpiness, then the so-called “effective size” of the chain is less, as indicated in the title of upper-middle
panel where it says “Eff.Sz. = 11723.9.”   
This is the equivalent number of values if they
were sampled independently of each other.   
The technical meaning of effective size will be discussed later in the chapter.

The left column of Figure 7.4 uses a relatively small proposal SD, namely $\theta = 0.02$, as indicated in the title of the upper-left panel where it says “Prpsl.SD = 0.02.”   
You can see that successive steps in the chain make small moves because the proposed jumps are small.   
In particular, in the lower-left panel you can see that it takes many steps for the chain to move away from the unrepresentative starting position of $\theta = 0.01$.   
The chain explores values only very gradually, producing a snake-like chain that lingers around particular values, thereby producing a form of clumpiness.  
In the long run, the chain will explore the posterior distribution thoroughly and produce a good representation, but it will require a very long chain. The title of the upper-left panel indicates that the effective size of this 50,000 step chain is only 468.9.

The right column of Figure 7.4 uses a relatively large proposal SD, namely $\theta = 2$, as indicated in the title of the upper-left panel where it says “Prpsl.SD = 2.”   
The proposed jumps are often far away from the bulk of the posterior distribution, and therefore, the proposals are often rejected and the chain stays at one value for many steps.   
The process accepts new values only occasionally, producing a very clumpy
chain.   
In the long run, the chain will explore the posterior distribution thoroughly
and produce a good representation, but it will require a very long chain.   
The title of the upper-right panel indicates that the effective size of this 50,000 step chain is  only 2113.4.

Regardless of the which proposal distribution in Figure 7.4 is used, the Metropolis algorithm will eventually produce an accurate representation of the posterior distribution, as is suggested by the histograms in the upper row of Figure 7.4.   
What differs is the efficiency of achieving a good approximation.   
The moderate proposal distribution will achieve a good approximation in fewer steps than either of the extreme proposal distributions.  
Later in the chapter, we will discuss criteria for deciding that a chain has
produced a sufficiently good approximation.

## 7.4. TOWARD GIBBS SAMPLING: ESTIMATING TWO COIN BIASES

The Metropolis method is very useful, but it can be inefficient.   
Other methods can be more efficient in some situations.   
In particular, another type of sampling method that can be very efficient is Gibbs sampling.   
Gibbs sampling typically applies to models with multiple parameters.

### 7.4.4. Gibbs sampling

The Metropolis algorithm is very general and broadly applicable.  
One problem with it, however, is that the proposal distribution must be properly tuned to the posterior distribution if the algorithm is to work well.   
If the proposal distribution is too narrow or too broad, a large proportion of proposed jumps will be rejected and the trajectory will
get bogged down in a localized region of the parameter space.   
Even at its most efficient, the effective size of the chain is far less than the number of proposed jumps.   
It would be nice, therefore, if we had another method of sample generation that was more efficient.  
Gibbs sampling can be especially useful for hierarchical models, which we explore in Chapter 9.   
It turns out that Gibbs sampling is a special case of the Metropolis-Hastings algorithm, which is a generalized form of the Metropolis algorithm.   

The procedure for Gibbs sampling is a type of random walk through parameter space, like the Metropolis algorithm.   
The walk starts at some arbitrary point, and at each point
in the walk, the next step depends only on the current position, and on no previous positions.  
What is different about Gibbs sampling, relative to the Metropolis algorithm, is how each step is taken.   
At each point in the walk, one of the component parameters
is selected.   
The component parameter could be selected at random, but typically the
parameters are cycled through, in order: $\theta_1, \theta_2, \theta_3, ... , \theta_1, \theta_2, \theta_3, ...$.   
The reason that parameters are cycled rather than selected randomly is that for complex models with many dozens or hundreds of parameters, it would take too many steps to visit every parameter by random chance alone, even though they would be visited about equally often in the long run.   
Suppose that parameter $\theta_i$ has been selected.  
Gibbs sampling then chooses a new value for that parameter by generating a random value directly from
the conditional probability distribution $p(\theta_i|\{\theta_{j\neq i}\}, D)$.   
The new value for $\theta_i$, combined with the unchanged values of $\theta_{j\neq i}$, constitutes the new position in the random walk.  
The process then repeats: Select a component parameter and select a new value for that parameter from its conditional posterior distribution.

<img style="float: left;" src="pic3/07_08.png"  width="500">

Figure 7.7 illustrates this process for a two-parameter example.    
In the first step, we want to select a new value for $\theta_1$.   
We conditionalize on the values of all the other
parameters, $\theta_{j\neq 1}$, from the previous step in the chain. In this example, there is only one other parameter, namely $\theta_2$.   
The upper panel of Figure 7.7 shows a slice through
the joint distribution at the current value of $\theta_2$.  
The heavy curve is the posterior distribution conditional on this value of $\theta_2$, denoted $p(\theta_1|\{\theta_{j\neq 1}\}, D)$, which is $p(\theta_i|\theta_2, D)$
in this case because there is only one other parameter.   
If the mathematical form of the conditional distribution is appropriate, a computer can directly generate a random
value of $\theta_1$.   
Having thereby generated a new value for $\theta_1$, we then conditionalize on it and determine the conditional distribution of the next parameter, $\theta_2$, as shown in the lower panel of Figure 7.7.   
The conditional distribution is denoted formally as
$p(\theta_2|\{\theta_{j\neq 2}\}, D)$, which equals $p(\theta_2|\theta_1, D)$ in this case of only two parameters.  
If the mathematical form is convenient, our computer can directly generate a random value of $\theta_2$ from the conditional distribution.   
We then conditionalize on the new value $\theta_2$, and the cycle repeats.

Gibbs sampling can be thought of as just a special case of the Metropolis algorithm, in which the proposal distribution depends on the location in parameter space and the component parameter selected.  
At any point, a component parameter is selected, and
then the proposal distribution for that parameter’s next value is the conditional posterior
probability of that parameter.  
Because the proposal distribution exactly mirrors the posterior
probability for that parameter, the proposed move is always accepted.   
A rigorous proof of this idea requires development of a generalized form of the Metropolis algorithm, called the
Metropolis-Hastings algorithm (Hastings, 1970). 

Gibbs sampling is especially useful when the complete joint posterior, $p(\{\theta_i\}|D)$, cannot be analytically determined and cannot be directly sampled, but all the conditional
distributions, $p(\theta_i|\{\theta_{j\neq i}\}, D)$, can be determined and directly sampled.  
